In [63]:
from langchain_classic.agents import create_react_agent
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

llm = ChatOllama(
    model="llama3.1:8b"
)
embeddings = OllamaEmbeddings(model="embeddinggemma")

# vector_store = InMemoryVectorStore(embeddings)

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)
vector_store._collection.count()

117

In [58]:
def add_document(path: str):
    loader = UnstructuredWordDocumentLoader(file_path=path)
    docs = loader.load()
    assert len(docs) == 1
    print(f"Total characters: {len(docs[0].page_content)}")

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # chunk size (characters)
        chunk_overlap=200,  # chunk overlap (characters)
        add_start_index=True,  # track index in original document
    )
    all_splits = text_splitter.split_documents(docs)
    print(f"Split blog post into {len(all_splits)} sub-documents.")

    document_ids = vector_store.add_documents(documents=all_splits)
    print(document_ids[:3])


In [59]:
add_document("../../data/Prepis_FG_PedagogLidr_mentori.docx")
add_document("../../data/Prepis_FG_PedagogLidr_SKUPINA-B-android.docx")

Total characters: 45890
Split blog post into 64 sub-documents.
['3307dfa4-f174-47bb-93e5-8d88b656dbc7', '6593c6fc-7c87-4fff-b2eb-915df145d67d', '4d546487-e49d-4f7e-829e-c44341222959']
Total characters: 39906
Split blog post into 53 sub-documents.
['f68aeaf1-bd51-4243-8794-92a4963020b3', 'f721ffe5-1874-4bcb-98bc-b9174f7282a2', 'a9ab3a3a-5ecb-429f-889f-d7ceb24ad698']


In [48]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs


In [49]:
from langchain.agents import create_agent

tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
     "You are a helpful assistant designed to answer questions using only information retrieved "
    "from local document excerpts. You have access to a single tool called `retrieve_context`, "
    "which can fetch relevant text from local documents based on a query.\n\n"
    "Guidelines:\n"
    "1. Always start by using `retrieve_context` with the user's full question.\n"
    "2. If the returned context does not appear relevant, or does not fully answer the question, "
    "call `retrieve_context` again with a refined or different query that could produce better results.\n"
    "3. You have to call the tool multiple times in a single reasoning process — it’s encouraged.\n"
    "4. If, after multiple attempts, you still cannot find sufficient context, clearly state that the "
    "information is not available in the local documents.\n\n"
    "Answer in Czech unless the user explicitly requests another language."
)
agent = create_agent(llm, tools, system_prompt=prompt)

In [53]:
query = (
    "Co uvedla mluvčí číslo 2 o klasifikačním řádu?"
    # "Vnímají respondenti nějakou zodpovědnost šířit ve své škole to, co se naučily v programu? Co na to říká učitel číslo 3"
    # "Jak často do školy docházela mentorka?"
    # "Kolik dětí má mluvčí 1?",
    # "Co by pedagogičtí lídři uvítaly na další rok programu?"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Co by pedagogičtí lídři uvítaly na další rok programu?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (02ee0872-acac-4e81-98db-e9713a85120b)
 Call ID: 02ee0872-acac-4e81-98db-e9713a85120b
  Args:
    query: Co by pedagogičiñi lůdri uvětali na dalsí rok programu
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': '../../data/Prepis_FG_PedagogLidr_SKUPINA-B-android.docx', 'start_index': 0}
Content: M – moderátor, pak očíslované mluvčí podle toho, jak odpovídaly na otázky. 

M: Co z programu Pedagog lídr považujete pro vaši učitelskou nebo ředitelskou praxi za nejužitečnější, nejdůležitější? Jaká témata z toho programu vás nejvíc oslovila a pokuste se prosím podívat i na začátek programu, nejen na poslední dva tři měsíce. 

1: Mě hodně oslovilo to plánování hodiny a vytyčen